#  OCR (Optical Character Recognition) - Implantação
## Utilização das bibliotecas [opencv](https://opencv.org/) e  [Tesseract OCR](https://tesseract-ocr.github.io/) para o reconhecimento de texto em imagens e da biblioteca [JiWER](https://github.com/jitsi/jiwer) para cálculo de mérticas de perfomance

*   Mais detlalhes sobre  o funcionamento dos algorítimos e das línguas nos quais o mesmo podem são utilizados são encontrados na [Tesseract documentation](https://tesseract-ocr.github.io/tessdoc/Data-Files)

*   Caso seja passado um arquivo .xlsx com as strings de target pode visualizar a perfonrmance do algorítimo

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
import base64
import cv2
from ocr import Class_Pytesseract_OCR


class Model:
    def __init__(self):
        self.loaded = False
    
    def load(self):
        # Following links explain why to use load() method insted of __init__()

        # Issue associated with seldon on __init__
        # https://github.com/SeldonIO/seldon-core/issues/2616

        # Solution for the case
        # https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html#gunicorn-and-load
        artifacts = joblib.load("/tmp/data/ocr.joblib")
        self.hyperparams = artifacts["hyperparams"]
        self.model_parameters = artifacts["model_parameters"]
        self.return_formats = artifacts["return_formats"]
        self.loaded = True

        
    def class_names(self):
        
        return ['bboxcoord1', 'bboxcoord2', 'bboxcoord3', 'bboxcoord4', 'text']
    
    def format_result(self, boxes, text):
        
        res = []
        
        for i in range(len(boxes)):
            
            bbox = list(map(float, boxes[i]))
            bbox.extend([text[i]])
            res.append(bbox)
        
        return res
    
    def predict(self, X, feature_names, meta=None):
        # First time load model
        if not self.loaded:
            self.load()
        
        #im_bytes = base64.b64decode(X[0])
        im_bytes = base64.b64decode(X[0,0])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        model = Class_Pytesseract_OCR(self.hyperparams,self.model_parameters)
        bboxes_or_image, text = model.predict(img, "Deployment", self.return_formats)
        
        if self.return_formats == "np_array":
            result = self.format_result(bboxes_or_image, text)
        else:
            result = [text, bboxes_or_image]
        
        return result